In [19]:
import tensorflow as tf
import numpy as np
import os

In [20]:
(train_images, train_labels), (test_images, test_labels) = tf.keras.datasets.mnist.load_data()

def preprocess_images(images):
    return images.reshape(-1, 784).astype(np.float32) / 255

def preprocess_labels(labels):
    return labels.reshape(-1).astype(np.int32)

train_images = preprocess_images(train_images)
test_images = preprocess_images(test_images)
train_labels = preprocess_labels(train_labels)
test_labels = preprocess_labels(test_labels)

train_data = tf.data.Dataset.from_tensor_slices((train_images, train_labels)).shuffle(60000).batch(128).repeat()
#test_data = tf.data.Dataset.from_tensor_slices((test_images, test_labels)).batch(128)

In [21]:
train_steps = 1000
lr = 0.1

W = tf.Variable(np.zeros([784, 10]).astype(np.float32))
b = tf.Variable(np.zeros(10, dtype=np.float32))

# first change: set up log dir and file writer(s)
from datetime import datetime
logdir = os.path.join("logs", "linear" + str(datetime.now()))
train_writer = tf.summary.create_file_writer(os.path.join(logdir, "train"))
test_writer = tf.summary.create_file_writer(os.path.join(logdir, "test"))

In [22]:
for step, (image_batch, label_batch) in enumerate(train_data):
    if step > train_steps:
        break

    with tf.GradientTape() as tape:
        logits = tf.matmul(image_batch, W) + b
        xent = tf.reduce_mean(tf.nn.sparse_softmax_cross_entropy_with_logits(
            logits=logits, labels=label_batch))
        
    grads = tape.gradient(xent, [W, b])
    W.assign_sub(lr * grads[0])
    b.assign_sub(lr * grads[1])
    
    # change #2: log this stuff every time step (rather wasteful)
    with train_writer.as_default():
        tf.summary.scalar("loss", xent, step=step)
        tf.summary.histogram("logits", logits, step=step)
        tf.summary.histogram("weights", W, step=step)
    
    if not step % 100:
        preds = tf.argmax(logits, axis=1, output_type=tf.int32)
        acc = tf.reduce_mean(tf.cast(tf.equal(preds, label_batch), tf.float32))
        
        # change #3: log this only once every 100 steps
        with train_writer.as_default():
            tf.summary.scalar("accuracy", acc, step=step)
            tf.summary.image("input", tf.reshape(image_batch, [-1, 28, 28, 1]), step=step)
            
        test_preds = tf.argmax(tf.matmul(test_images, W) + b, axis=1, output_type=tf.int32)
        test_acc = tf.reduce_mean(tf.cast(tf.equal(test_preds, test_labels), tf.float32))

        
        print("Training Loss: {} Accuracy: {}".format(xent, acc))
        print("Test accuracy: {}\n".format(test_acc))

Training Loss: 2.3025851249694824 Accuracy: 0.109375
Test accuracy: 0.3434999883174896

Training Loss: 0.6413254737854004 Accuracy: 0.7890625
Test accuracy: 0.8689000010490417

Training Loss: 0.44259291887283325 Accuracy: 0.8984375
Test accuracy: 0.885699987411499

Training Loss: 0.395376056432724 Accuracy: 0.9140625
Test accuracy: 0.8907999992370605

Training Loss: 0.4814039170742035 Accuracy: 0.8671875
Test accuracy: 0.8962999582290649

Training Loss: 0.3444076180458069 Accuracy: 0.921875
Test accuracy: 0.8999999761581421

Training Loss: 0.4626608192920685 Accuracy: 0.8828125
Test accuracy: 0.9021999835968018

Training Loss: 0.3043449819087982 Accuracy: 0.921875
Test accuracy: 0.9059999585151672

Training Loss: 0.4015709161758423 Accuracy: 0.8984375
Test accuracy: 0.9061999917030334

Training Loss: 0.34627628326416016 Accuracy: 0.921875
Test accuracy: 0.9088000059127808

Training Loss: 0.3838801980018616 Accuracy: 0.8828125
Test accuracy: 0.9095999598503113



In [23]:
# then load/run tensorboard

%load_ext tensorboard

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


In [24]:
%tensorboard --logdir logs

Reusing TensorBoard on port 6006 (pid 93749), started 1:15:07 ago. (Use '!kill 93749' to kill it.)